In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py

In [2]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

In [3]:
data_loc = '/gpfs/slac/atlas/fs1/d/rafaeltl/public/ML/L1RNN/datasets_2020_ff/'

In [4]:
file_str = 'Jan06_FlavFix_smear_1_std_xtd_zst.h5'

In [5]:
f5 = h5py.File(data_loc+file_str, 'r')

In [6]:
x_train = np.array( f5['x_train'] )
y_train = to_categorical ( np.array( f5['y_train'] ) )
w_train = np.array( f5['w_train'] )

In [7]:
y_train

array([[0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.]], dtype=float32)

In [8]:
from tensorflow.keras.layers import Dense, Activation, BatchNormalization, LSTM, Masking, Input, GRU
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1

In [9]:
from tensorflow.keras import regularizers

In [10]:
def lstmmodel(max_len, n_var, rec_units, ndense=[10], l1_reg=0,
              l2_reg=0, rec_act='sigmoid', extra_lab='none', rec_kernel_init='VarianceScaling',
             dense_kernel_init='lecun_uniform', domask=False):
    
    rec_layer = 'LSTM'
    
    track_inputs = Input(shape=(max_len, n_var,))
    
    if domask:
        hidden = Masking( mask_value=0, name="masking_1")(track_inputs)
    else:
        hidden = track_inputs
    
    if l1_reg > 1e-6 and l2_reg > 1e-6:
        hidden = LSTM(units=rec_units,
                  recurrent_activation = rec_act,
                  kernel_initializer = rec_kernel_init, 
                  kernel_regularizer = regularizers.l1_l2(l1 = l1_reg, l2 = l2_reg),
                  name = 'lstm1_l1l2')(hidden)
    elif l1_reg > 1e-6:
        hidden = LSTM(units=rec_units,
                  recurrent_activation = rec_act,
                  kernel_initializer = rec_kernel_init, 
                  kernel_regularizer = regularizers.l1(l1 = l1_reg),
                  name = 'lstm1_l1')(hidden)
    elif l2_reg > 1e-6:
        hidden = LSTM(units=rec_units,
                  recurrent_activation = rec_act,
                  kernel_initializer = rec_kernel_init, 
                  kernel_regularizer = regularizers.l2(l2 = l2_reg),
                  name = 'lstm1_l2')(hidden)
    else:
        hidden = LSTM(units=rec_units,
                  recurrent_activation = rec_act,
                  kernel_initializer = rec_kernel_init, 
                  name = 'lstm1')(hidden)

    for ind,nd in enumerate(ndense):
        hidden = Dense(nd, activation='relu', kernel_initializer=dense_kernel_init, name=f'dense_{ind}' )(hidden)
    
    output = Dense(3, activation='softmax', kernel_initializer=dense_kernel_init, name = 'output_softmax')(hidden)
    
    model = Model(inputs=track_inputs, outputs=output)
    
    d_layers = ''.join([ str(dl) for dl in ndense ])
        
    if domask:
        mname  = f'MASKED_rnn_{rec_layer}.{rec_units}_Dense.{d_layers}_'
    else:
        mname  = f'rnn_{rec_layer}.{rec_units}_Dense.{d_layers}_'
    mname += f'LSTMKernelInit.{rec_kernel_init}_DenseKernelInit.{dense_kernel_init}'
    mname += f'KRl1.{l1_reg}_KRl2.{l2_reg}_recAct.{rec_act}' #LSTM kernel regularizer
    
    if 'none' not in extra_lab:
        mname += f'_{extra_lab}'
    
    return model, mname

#     mask = Masking( mask_value=0, name="masking_1")(track_inputs)
##########################################
#                   use_bias=False,
#                   activation='relu',
#                   recurrent_activation='relu',
#                   kernel_regularizer = regularizers.l1_l2(l1= 0.001, l2 = 0.0001), 
#                   bias_regularizer = regularizers.l1_l2(l1= 1, l2 = 1), 
#                   activity_regularizer=regularizers.l1_l2(l1= 0.001, l2 = 0.0001),
##########################################


In [11]:
def grumodel(max_len, n_var, rec_units, ndense=[10], l1_reg=0,
              l2_reg=0, rec_act='sigmoid', extra_lab='none', rec_kernel_init='VarianceScaling',
             dense_kernel_init='lecun_uniform', domask=False):
    
    rec_layer = 'GRU'
    
    track_inputs = Input(shape=(max_len, n_var,))
    
    if domask:
        hidden = Masking( mask_value=0, name="masking_1")(track_inputs)
    else:
        hidden = track_inputs
    

    if l1_reg > 1e-6 and l2_reg > 1e-6:
        hidden = GRU(units=rec_units,
                  kernel_initializer = rec_kernel_init, 
                  kernel_regularizer = regularizers.l1_l2(l1 = l1_reg, l2 = l2_reg),
                  name = 'gru_l1l2')(hidden)
    elif l1_reg > 1e-6:
        hidden = GRU(units=rec_units,
                  recurrent_activation = rec_act,
                  kernel_initializer = rec_kernel_init, 
                  kernel_regularizer = regularizers.l1(l1 = l1_reg),
                  name = 'gru_l1')(hidden)
    elif l2_reg > 1e-6:
        hidden = GRU(units=rec_units,
                  recurrent_activation = rec_act,
                  kernel_initializer = rec_kernel_init, 
                  kernel_regularizer = regularizers.l2(l2 = l2_reg),
                  name = 'gru_l2')(hidden)
    else:
        hidden = GRU(units=rec_units,
                  recurrent_activation = rec_act,
                  kernel_initializer = rec_kernel_init, 
                  name = 'gru')(hidden)
            

    for ind,nd in enumerate(ndense):
        hidden = Dense(nd, activation='relu', kernel_initializer=dense_kernel_init, name=f'dense_{ind}' )(hidden)
    
    output = Dense(3, activation='softmax', kernel_initializer=dense_kernel_init, name = 'output_softmax')(hidden)
    
    model = Model(inputs=track_inputs, outputs=output)
    
    d_layers = ''.join([ str(dl) for dl in ndense ])
        
    if domask:
        mname  = f'MASKED_rnn_{rec_layer}.{rec_units}_Dense.{d_layers}_'
    else:
        mname  = f'rnn_{rec_layer}.{rec_units}_Dense.{d_layers}_'
    mname += f'LSTMKernelInit.{rec_kernel_init}_DenseKernelInit.{dense_kernel_init}'
    mname += f'KRl1.{l1_reg}_KRl2.{l2_reg}_recAct.{rec_act}' #LSTM kernel regularizer
    
    if 'none' not in extra_lab:
        mname += f'_{extra_lab}'
    
    return model, mname

#     mask = Masking( mask_value=0, name="masking_1")(track_inputs)
##########################################
#                   use_bias=False,
#                   activation='relu',
#                   recurrent_activation='relu',
#                   kernel_regularizer = regularizers.l1_l2(l1= 0.001, l2 = 0.0001), 
#                   bias_regularizer = regularizers.l1_l2(l1= 1, l2 = 1), 
#                   activity_regularizer=regularizers.l1_l2(l1= 0.001, l2 = 0.0001),
##########################################


In [12]:
l1_reg = 0
l2_reg = 0

## GRU Model

# model, model_name = grumodel(15, 6, 120, [50, 10], l1_reg=l1_reg, l2_reg=l2_reg)

## LSTM Model

model, model_name = lstmmodel(15, 6, 120, [50, 10], l1_reg=l1_reg, l2_reg=l2_reg)

# Masked model

# model, model_name = lstmmodel(15, 6, 20, [10], l1_reg=l1_reg, l2_reg=l2_reg, domask=True)


# ## Very very tiny model

# model, model_name = lstmmodel(15, 6, 2, [], l1_reg=l1_reg, l2_reg=l2_reg)

# ## Very tiny model

# model, model_name = lstmmodel(15, 6, 10, [], l1_reg=l1_reg, l2_reg=l2_reg)

# ## Tiny model

# model, model_name = lstmmodel(15, 6, 10, [10], l1_reg=l1_reg, l2_reg=l2_reg)

# ## Small model

# model, model_name = lstmmodel(15, 6, 20, [10], l1_reg=l1_reg, l2_reg=l2_reg)

# ## Little model

# model, model_name = lstmmodel(15, 6, 50, [10], l1_reg=l1_reg, l2_reg=l2_reg)

# ## Intermediate model

# model, model_name = lstmmodel(15, 6, 50, [10, 10], l1_reg=l1_reg, l2_reg=l2_reg)

# ## Large model

# model, model_name = lstmmodel(15, 6, 100, [50, 10], l1_reg=l1_reg, l2_reg=l2_reg)

# model, model_name = lstmmodel(15, 6, 100, [10], l1_reg=l1_reg, l2_reg=l2_reg)

# model, model_name = lstmmodel(15, 6, 100, [10], l1_reg=l1_reg, l2_reg=l2_reg)

In [13]:
model.summary()
print(model_name)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 15, 6)]           0         
_________________________________________________________________
lstm1 (LSTM)                 (None, 120)               60960     
_________________________________________________________________
dense_0 (Dense)              (None, 50)                6050      
_________________________________________________________________
dense_1 (Dense)              (None, 10)                510       
_________________________________________________________________
output_softmax (Dense)       (None, 3)                 33        
Total params: 67,553
Trainable params: 67,553
Non-trainable params: 0
_________________________________________________________________
rnn_LSTM.120_Dense.5010_LSTMKernelInit.VarianceScaling_DenseKernelInit.lecun_uniformKRl1.0_KRl2.0_recAct.sigmoid


In [ ]:
model_json = model.to_json()
with open(f'keras/model_{model_name}_arch.json', "w") as json_file:
    json_file.write(model_json)

In [ ]:
# adam = Adam(learning_rate=0.01)
model.compile(optimizer='adam', loss=['categorical_crossentropy'], metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
model_output = f'keras/model_{model_name}_weights.h5'

In [ ]:
train = True

In [ ]:
if train:
    history = model.fit( x_train , y_train,
            batch_size=2**14,
            # epochs=10,
            epochs=150,
            validation_split=0.1,
            shuffle = True,
            sample_weight= w_train,
            callbacks = [
                EarlyStopping(verbose=True, patience=20, monitor='val_accuracy'),
                ModelCheckpoint(model_output, monitor='val_accuracy', verbose=True, save_best_only=True)
                ],
            verbose=True
            )
    
model.load_weights(model_output)

In [ ]:
model.summary()

In [ ]:
x_test = np.array( f5['x_test'] )
y_test = to_categorical ( np.array( f5['y_test'] ) )

In [ ]:
import plotting
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

In [ ]:
import importlib

In [ ]:
importlib.reload(plotting)

In [ ]:
pred_test = model.predict(x_test, batch_size=2**10)

In [ ]:
print("Accuracy: {}".format(accuracy_score(np.argmax(y_test, axis=1), np.argmax(pred_test, axis=1))))

In [ ]:
pb_b = pred_test[:,0] [y_test[:,0] == 1]
pc_b = pred_test[:,1] [y_test[:,0] == 1]
pl_b = pred_test[:,2] [y_test[:,0] == 1]
    
pc_c = pred_test[:,1] [y_test[:,1] == 1]
pb_c = pred_test[:,0] [y_test[:,1] == 1]
    
pl_l = pred_test[:,2] [y_test[:,2] == 1]
pb_l = pred_test[:,0] [y_test[:,2] == 1]

plt.Figure()

plt.hist( pb_b/(pb_b+pl_b), range=(0,1), bins=1000, histtype='step' )
plt.hist( pb_l/(pb_l+pl_l), range=(0,1), bins=1000, histtype='step' )

plt.show()


plt.Figure()

plt.hist( pb_b/(pb_b+pc_b), range=(0,1), bins=1000, histtype='step' )
plt.hist( pb_c/(pb_c+pc_c), range=(0,1), bins=1000, histtype='step' )

plt.show()

In [ ]:
plt.figure(figsize=(9,9))
_ = plotting.makeRoc(y_test, pred_test)

In [ ]:
for layer in model.layers:
    print(layer.name)
#     plt.Figure()
    
    this_wgts = layer.get_weights()
#     if len(this_wgts) < 1: continue
    print(layer.get_config())
    
    for wgt in this_wgts:
        print(wgt)
        print()
#     max_wgts = np.max(this_wgts)
#     min_wgts = np.min(this_wgts)
#     plt.hist(this_wgts, bins=100, range=(min_wgts, max_wgts))
#     plt.xlabel(f'{layer.name}')
#     plt.show